In [1]:
#%pip install flask

  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)

   ---------------------------------------- 0/6 [werkzeug]
   ---------------------------------------- 0/6 [werkzeug]
   ---------------------------------------- 0/6 [werkzeug]
   ---------------------------------------- 0/6 [werkzeug]
   ---------------------------------------- 0/6 [werkzeug]
   ---------------------------------------- 0/6 [werkzeug]
   ---------------------------------------- 0/6 [werkzeug]
   ---------------------------------------- 0/6 [werkzeug]
   ---------------------------------------- 0/6 [w

In [2]:
#%pip install werkzeug

Note: you may need to restart the kernel to use updated packages.


In [3]:
#%pip install ipython

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import sqlite3
from flask import Flask, jsonify, request, render_template_string
from threading import Thread
from werkzeug.serving import make_server
from IPython.display import display, HTML
from datetime import datetime # Importar datetime para timestamps

# Inicializa o Flask
app = Flask(__name__)

# Caminho para o banco
BASE_DIR = os.getcwd()
DB_FILE = os.path.join(BASE_DIR, "chatbot_pizzaria.db")

# Criação e povoamento do banco de dados
def init_db():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    # Tabela para o chat (perguntas/respostas fixas)
    c.execute('''
        CREATE TABLE IF NOT EXISTS chat (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            pergunta TEXT NOT NULL UNIQUE,
            resposta TEXT NOT NULL
        )
    ''')
    # Nova tabela para reclamações
    c.execute('''
        CREATE TABLE IF NOT EXISTS complaints (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_complaint TEXT NOT NULL,
            timestamp TEXT NOT NULL
        )
    ''')
    exemplos = [
        ("olá", "Olá! Como posso ajudar você hoje? 😊"),
        ("cardápio online", "Confira nosso cardápio completo em: <a href='https://anota.ai/blog/baixar-anota-ai/' target='_blank' style='color: #004aad; text-decoration: underline; font-weight: bold;'>Clique aqui!</a>"),
        ("promoção do dia", "A promoção de hoje é: Pizza Grande de Calabresa por R$ 35,00! 🍕 Corre la no nosso cardápio online e faça seu pedido!"),
        ("pedido", "Certo! Vamos verificar o status do seu pedido. Por favor, aguarde um momento..."),
        ("reclamações", "Lamentamos qualquer inconveniente. Por favor, descreva sua reclamação para que possamos ajudar."),
        ("encerrar chat", "Foi um prazer ajudar! Se precisar de algo mais, é só abrir o chat novamente. 😊"),
        ("resposta_reclamacao_padrao", "Obrigado por nos informar! Sua reclamação foi registrada. Seu feedback é muito importante para nós."),
    ]
    for pergunta, resposta in exemplos:
        try:
            c.execute("INSERT INTO chat (pergunta, resposta) VALUES (?, ?)", (pergunta, resposta))
        except sqlite3.IntegrityError:
            continue
    conn.commit()
    conn.close()

init_db()

# HTML do site com o chatbot embutido
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="pt-BR">
<head>
<meta charset="UTF-8" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<title>Pizzaria - Chatbot</title>
<style>
    body { font-family: 'Segoe UI', sans-serif; background: #f4f6f8; margin: 0; }
    header, footer { background-color: #004aad; color: white; text-align: center; padding: 20px; }
    main { padding: 40px; max-width: 800px; margin: auto; }
    h1 { color: #004aad; }
    #chatbot-toggle {
        position: fixed;
        bottom: 25px;
        right: 25px;
        width: 60px; height: 60px;
        border-radius: 50%;
        background: #004aad;
        display: flex; justify-content: center; align-items: center;
        cursor: pointer;
        z-index: 999;
    }
    #chatbot-toggle svg { width: 30px; height: 30px; fill: white; }

    #chatbot-box {
        position: fixed;
        bottom: 100px;
        right: 25px;
        width: 350px;
        max-height: 500px;
        background: white;
        border-radius: 12px;
        box-shadow: 0 4px 12px rgba(0,0,0,0.3);
        display: none;
        flex-direction: column;
        overflow: hidden;
        z-index: 1000;
    }
    #chatbot-header {
        background: #004aad;
        color: white;
        padding: 12px;
        font-weight: bold;
        display: flex;
        justify-content: space-between;
    }
    #chat-log {
        padding: 10px;
        overflow-y: auto; /* Garante a barra de rolagem */
        flex-grow: 1;
        background: #f1f3f5;
        display: flex;
        flex-direction: column;
        gap: 10px;
    }
    .message {
        max-width: 80%;
        padding: 10px;
        border-radius: 12px;
        font-size: 14px;
    }
    .user { align-self: flex-end; background: #0d6efd; color: white; }
    .bot { align-self: flex-start; background: #dee2e6; }
    /* Estilo para links dentro das mensagens do bot, se definidos inline ou sobrescritos */
    .bot a {
        color: #004aad; /* Cor do link */
        text-decoration: underline;
        font-weight: bold;
    }
    .bot a:hover {
        color: #003a8d;
    }


    /* Estilo para a caixa de botões dentro do chat-log */
    .menu-message {
        align-self: flex-start;
        background: #dee2e6;
        border-radius: 12px;
        padding: 10px;
        max-width: 80%;
        font-size: 14px;
        display: flex;
        flex-direction: column;
        gap: 5px;
    }

    .menu-message .menu-title {
        margin-bottom: 5px;
        font-weight: bold;
    }

    .menu-message button {
        margin: 0;
        padding: 8px 12px;
        border-radius: 15px;
        border: none;
        background: #004aad;
        color: white;
        cursor: pointer;
        font-size: 14px;
        text-align: center;
        width: 100%;
        box-sizing: border-box;
    }
    .menu-message button:hover {
        background: #003a8d;
    }

    /* Estilo para a mensagem de status do pedido consolidado */
    .consolidated-order-status {
        align-self: flex-start;
        background: #dee2e6;
        border-radius: 12px;
        padding: 10px;
        max-width: 80%;
        font-size: 14px;
        margin-bottom: 5px;
    }
    .consolidated-order-status strong {
        color: #004aad;
    }
    .consolidated-order-status ul {
        list-style: none;
        padding: 0;
        margin-top: 5px;
    }
    .consolidated-order-status li {
        margin-bottom: 3px;
    }


    #input-area {
        display: flex;
        padding: 10px;
        border-top: 1px solid #ccc;
        background: #fff;
    }
    #user-input {
        flex-grow: 1;
        padding: 8px;
        border: 1px solid #ccc;
        border-radius: 20px;
        font-size: 14px;
    }
    .send-btn {
        margin-left: 10px;
        padding: 8px 14px;
        background: #004aad;
        color: white;
        border: none;
        border-radius: 20px;
        cursor: pointer;
    }
</style>
</head>
<body>

<header>Pizzaria - Chatbot</header>
<main>
    <h1>Bem-vindo(a)  Pizzaria do Cheff!</h1>
    <p>Use o chatbot no canto inferior direito para tirar suas dúvidas sobre nosso cardápio, promoções e muito mais!</p>
</main>
<footer>&copy; 2025 Pizzaria do Cheff </footer>

<div id="chatbot-toggle" onclick="toggleChatbot()">
    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 64 64">
        <circle cx="32" cy="32" r="30" fill="#004aad"/>
        <rect x="18" y="24" width="8" height="8" rx="2" fill="#fff"/>
        <rect x="38" y="24" width="8" height="8" rx="2" fill="#fff"/>
    </svg>
</div>

<div id="chatbot-box">
    <div id="chatbot-header">
        Chatbot 🤖
        <button onclick="toggleChatbot()">×</button>
    </div>
    <div id="chat-log"></div>
    <div id="input-area">
        <input type="text" id="user-input" placeholder="Digite sua mensagem..." />
        <button class="send-btn" onclick="sendMessage()">Enviar</button>
    </div>
</div>

<script>
    const chatbotBox = document.getElementById('chatbot-box');
    const chatLog = document.getElementById('chat-log');
    const userInput = document.getElementById('user-input');
    const sendBtn = document.querySelector('.send-btn');

    let hasGreeted = false;
    let expectingComplaintInput = false; // Estado para reclamação

    // Novos estados e variáveis para coleta de dados do cliente
    let collectingInfo = false;
    let currentInfoStep = 0; // 0: Nenhum, 1: Nome, 2: Telefone, 3: Endereço
    let customerName = '';
    let customerPhone = '';
    let customerAddress = '';

    const menuOptions = [
        { text: 'Cardápio online', value: 'cardápio online' },
        { text: 'Promoção do dia', value: 'promoção do dia' },
        { text: 'Pedido', value: 'pedido' },
        { text: 'Reclamações', value: 'reclamações' },
        { text: 'Encerrar Chat', value: 'encerrar chat', isSpecial: true }
    ];

    function appendMessage(text, sender) {
        const div = document.createElement('div');
        div.className = 'message ' + sender;
        // Usa innerHTML para mensagens do bot para renderizar HTML
        if (sender === 'bot') {
            div.innerHTML = text;
        } else {
            div.innerText = text; // Usa innerText para mensagens do usuário por segurança (previne XSS)
        }
        chatLog.appendChild(div);
        chatLog.scrollTop = chatLog.scrollHeight;
    }

    function appendMenuMessage(title, options) {
        const menuDiv = document.createElement('div');
        menuDiv.className = 'menu-message bot';

        const titleSpan = document.createElement('span');
        titleSpan.className = 'menu-title';
        titleSpan.innerText = title;
        menuDiv.appendChild(titleSpan);

        options.forEach(option => {
            const button = document.createElement('button');
            button.innerText = option.text;
            button.onclick = () => handleMenuButtonClick(option.value, option.isSpecial);
            menuDiv.appendChild(button);
        });

        chatLog.appendChild(menuDiv);
        chatLog.scrollTop = chatLog.scrollHeight;
    }

    // Função para controlar a ativação/desativação do input e botões
    function setInputActive(active) {
        userInput.disabled = !active;
        sendBtn.disabled = !active;
    }

    function startCollectingInfo() {
        collectingInfo = true;
        currentInfoStep = 1; // Começa com o nome
        setInputActive(true); // Ativa o input
        appendMessage('Olá! Para começarmos, qual é o seu nome completo?', 'bot');
        userInput.focus(); // Coloca o foco no input
    }

    function processCollectedInfo(msg) {
        if (currentInfoStep === 1) {
            // Validação para o nome: agora verifica se contém letras e espaços, e não apenas números ou caracteres especiais.
            // Permite letras (maiúsculas/minúsculas), espaços e acentos comuns em nomes (à, á, â, ã, é, ê, í, ó, ô, õ, ú, ü, ç).
            const nameRegex = /^[A-Za-z\u00C0-\u00FF\s]+$/; // regex para letras, espaços e acentos
            if (!nameRegex.test(msg)) {
                appendMessage('Por favor, digite um nome válido, usando apenas letras e espaços.', 'bot');
            } else if (msg.length < 2) {
                appendMessage('Por favor, digite um nome mais completo.', 'bot');
            } else {
                customerName = msg;
                appendMessage(`Ok, ${customerName}!`, 'bot');
                currentInfoStep = 2;
                appendMessage('Qual é o seu número de telefone (com DDD), por favor?', 'bot');
            }
        } else if (currentInfoStep === 2) {
            // ... (restante do código de validação do telefone, que já estava OK) ...
            if (!/^\d{10,11}$/.test(msg)) {
                appendMessage('Por favor, digite um número de telefone válido (apenas números, com DDD).', 'bot');
            } else {
                customerPhone = msg;
                appendMessage('Certo, telefone salvo!', 'bot');
                currentInfoStep = 3;
                appendMessage('E qual o seu endereço completo (rua, número, bairro, cidade)?', 'bot');
            }
        } else if (currentInfoStep === 3) {
            // Validação para o endereço:
            // - Verifica se não é apenas números.
            // - Verifica se tem um comprimento razoável (mais de 10 caracteres para ser mais específico).
            // - Verifica se contém pelo menos uma letra (para evitar endereços apenas com símbolos ou números).
            const hasLetters = /[A-Za-z\u00C0-\u00FF]/.test(msg); // Verifica se tem pelo menos uma letra (incluindo acentos)
            const isMostlyNumbers = /^\d+$/.test(msg.trim()); // Verifica se é *apenas* números (após remover espaços)

            if (isMostlyNumbers || !hasLetters || msg.length < 10) { // Alterado o comprimento mínimo para 10
                appendMessage('Por favor, digite um endereço mais completo e válido (rua, número, bairro, cidade).', 'bot');
            } else {
                customerAddress = msg;
                appendMessage('Endereço registrado. Obrigado!', 'bot');
                collectingInfo = false;
                currentInfoStep = 0; // Reset
                appendMessage('Agora, como posso ajudar você?', 'bot');
                appendMenuMessage('Selecione uma opção:', menuOptions);
            }
        }
        userInput.focus();
    }


    function showConsolidatedOrderStatus() {
        // Usando as informações coletadas do cliente
        const customerOrder = {
            id: 'ABCDE-12345',
            status: 'Em preparo',
            estimatedTime: '25-30 min',
            items: ['Pizza Calabresa G (sem cebola).......R$ 52,00', 'Coca-Cola 2L.......R$ 9,70', 'Batata Frita P.......R$ 14,00'],
            deliveryAddress: customerAddress || 'Endereço não informado',
            customerInfo: `${customerName || 'Cliente'} (${customerPhone || 'Telefone não informado'})`
        };

        appendMessage('Buscando informações do seu pedido...', 'bot');

        setTimeout(() => {
            if (customerOrder) {
                let itemsList = customerOrder.items.map(item => `<li>${item}</li>`).join('');

                let orderHtml = `<div class="consolidated-order-status bot">
                    <strong>Status do Pedido #${customerOrder.id}</strong><br>
                    <ul>
                        <li>Status Atual: <strong>${customerOrder.status}</strong></li>
                        <li>Tempo estimado de entrega: ${customerOrder.estimatedTime}</li>
                        <li>Itens:</li>
                        <ul>
                            ${itemsList}
                        </ul>
                        <li>Valor do pedido: R$ 75,70 </li>
                        <li>Valor da entrega: R$ 10,00 </li><br>
                        <li>Valor Total:R$ 85,70 </li><br>
                        <li>Endereço de Entrega: ${customerOrder.deliveryAddress}</li>
                        <li>Contato: ${customerOrder.customerInfo}</li>
                    </ul>
                    Agradecemos a sua paciência!
                </div>`;
                chatLog.innerHTML += orderHtml;
                chatLog.scrollTop = chatLog.scrollHeight;
                appendMenuMessage('Selecione outra opção:', menuOptions);
            } else {
                appendMessage('Não encontramos nenhum pedido ativo associado ao seu ID de cliente.', 'bot');
                appendMenuMessage('Selecione outra opção:', menuOptions);
            }
            userInput.focus();
        }, 1500);
    }

    function handleMenuButtonClick(value, isSpecial) {
        setInputActive(false);

        appendMessage(value, 'user');

        expectingComplaintInput = false;

        if (isSpecial && value === 'encerrar chat') {
            fetch('/chat', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({ pergunta: value })
            })
            .then(res => res.json())
            .then(data => {
                appendMessage(data.resposta, 'bot');
                setTimeout(() => {
                    toggleChatbot();
                    chatLog.innerHTML = '';
                    hasGreeted = false;
                    customerName = '';
                    customerPhone = '';
                    customerAddress = '';
                    currentInfoStep = 0;
                    collectingInfo = false;
                    setInputActive(false);
                }, 1500);
            }).catch(error => {
                console.error('Erro ao encerrar chat:', error);
                appendMessage('Desculpe, ocorreu um erro ao encerrar o chat.', 'bot');
                setInputActive(true);
            });
        } else if (value === 'pedido') {
            showConsolidatedOrderStatus();
        } else if (value === 'reclamações') {
            expectingComplaintInput = true;
            fetch('/chat', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({ pergunta: value })
            })
            .then(res => res.json())
            .then(data => {
                appendMessage(data.resposta, 'bot');
                setInputActive(true);
                userInput.focus();
            }).catch(error => {
                console.error('Erro ao solicitar reclamação:', error);
                appendMessage('Desculpe, ocorreu um erro ao iniciar o processo de reclamação.', 'bot');
                setInputActive(true);
            });
        }
        else { // Para todas as outras opções de menu
            fetch('/chat', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({ pergunta: value })
            })
            .then(res => res.json())
            .then(data => {
                appendMessage(data.resposta, 'bot');
                appendMenuMessage('Selecione outra opção:', menuOptions);
                setInputActive(true);
                userInput.focus();
            }).catch(error => {
                console.error('Erro ao processar opção do menu:', error);
                appendMessage('Desculpe, ocorreu um erro ao processar sua seleção.', 'bot');
                setInputActive(true);
            });
        }
    }

    function toggleChatbot() {
        if (chatbotBox.style.display === 'flex') {
            chatbotBox.style.display = 'none';
        } else {
            chatbotBox.style.display = 'flex';
            if (!hasGreeted) {
                startCollectingInfo();
                hasGreeted = true;
            } else {
                setInputActive(true);
                if (!collectingInfo && currentInfoStep === 0) {
                    appendMessage('Olá novamente! Como posso ajudar você?', 'bot');
                    appendMenuMessage('Selecione uma opção:', menuOptions);
                } else if (collectingInfo) {
                    if (currentInfoStep === 1) {
                        appendMessage('Ainda estou esperando seu nome completo, por favor.', 'bot');
                    } else if (currentInfoStep === 2) {
                        appendMessage('Ainda estou esperando seu número de telefone, por favor.', 'bot');
                    } else if (currentInfoStep === 3) {
                        appendMessage('Ainda estou esperando seu endereço completo, por favor.', 'bot');
                    }
                }
                userInput.focus();
            }
        }
    }

    function sendMessage() {
        const msg = userInput.value.trim();
        if (!msg) return;

        appendMessage(msg, 'user');
        userInput.value = ''; // Limpa o input imediatamente

        if (collectingInfo) {
            processCollectedInfo(msg); // Processa a informação do passo atual
            // userInput.focus(); // Já é chamado dentro de processCollectedInfo
            return; // Sai da função, não envia para o Flask ainda
        }

        let payload;
        if (expectingComplaintInput) {
            payload = { pergunta: msg, is_complaint: true };
            expectingComplaintInput = false; // Reseta o estado após enviar a reclamação
        } else {
            payload = { pergunta: msg };
        }

        setInputActive(false); // Desativa input e botão durante a requisição

        fetch('/chat', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify(payload)
        })
        .then(res => {
            if (!res.ok) { // Verifica se a resposta HTTP foi bem-sucedida (status 2xx)
                throw new Error(`Erro de rede ou servidor: ${res.status}`);
            }
            return res.json();
        })
        .then(data => {
            appendMessage(data.resposta, 'bot');
            // Reexibir o menu após uma resposta do bot, a menos que seja para encerrar
            // Ou se a mensagem acabou de ser uma reclamação enviada e a resposta padrão foi dada.
            if (payload.pergunta !== 'encerrar chat') {
                appendMenuMessage('Selecione outra opção:', menuOptions);
            }
            setInputActive(true); // Reativa input e botão após a resposta do bot
            userInput.focus(); // Coloca o foco no input
        })
        .catch(error => {
            console.error('Erro ao comunicar com o servidor:', error);
            appendMessage('Desculpe, ocorreu um erro ao processar sua solicitação. Por favor, tente novamente.', 'bot');
            setInputActive(true); // Reativa input e botão mesmo com erro
            userInput.focus(); // Coloca o foco no input
        });
    }

    // Adiciona o event listener para a tecla 'Enter'
    userInput.addEventListener('keydown', function(event) {
        if (event.key === 'Enter') {
            sendMessage();
            event.preventDefault(); // Previne a quebra de linha no input
        }
    });

    window.onload = function() {
        chatbotBox.style.display = 'none';
        // Garante que o input está desativado até o chatbot ser aberto e iniciar a coleta
        setInputActive(false);
    };
</script>

</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json # Captura todos os dados enviados
    pergunta = data.get("pergunta", "").lower().strip()
    is_complaint = data.get("is_complaint", False) # Captura o flag is_complaint

    resposta = "Desculpe, não entendi sua pergunta. 🤔" # Resposta padrão
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    if is_complaint: # Verifica se é uma reclamação
        user_complaint = pergunta # A 'pergunta' é a reclamação do usuário neste caso
        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        try:
            c.execute("INSERT INTO complaints (user_complaint, timestamp) VALUES (?, ?)", (user_complaint, current_timestamp))
            conn.commit()
            # Busca a resposta padrão de reclamação do banco de dados
            c.execute("SELECT resposta FROM chat WHERE LOWER(pergunta)='resposta_reclamacao_padrao'")
            row = c.fetchone()
            if row:
                resposta = row[0] # Define a resposta como a mensagem padrão do banco
            else:
                resposta = "Sua reclamação foi registrada, mas não consegui encontrar a mensagem de confirmação. Entraremos em contato!"
        except sqlite3.Error as e:
            print(f"Erro ao inserir reclamação: {e}")
            resposta = "Desculpe, não foi possível registrar sua reclamação no momento. Por favor, tente novamente mais tarde."
    else: # Se não for uma reclamação, processa como pergunta normal
        c.execute("SELECT resposta FROM chat WHERE LOWER(pergunta)=?", (pergunta,))
        row = c.fetchone()
        if row:
            resposta = row[0]
        # Se a pergunta não for encontrada e não for uma reclamação em andamento,
        # a 'resposta' mantém o valor padrão "Desculpe, não entendi...".

    conn.close()
    return jsonify({"resposta": resposta})

# Rodar o servidor no Jupyter Notebook
class ServerThread(Thread):
    def __init__(self, app):
        Thread.__init__(self)
        self.srv = make_server('0.0.0.0', 5005, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        self.srv.serve_forever()

    def shutdown(self):
        self.srv.shutdown()

server = ServerThread(app)
server.start()

display(HTML("<a href='http://localhost:5005' target='_blank'>🔗 Clique aqui para abrir o site com o chatbot</a>"))

127.0.0.1 - - [07/Jun/2025 19:35:46] "GET / HTTP/1.1" 200 -


AssertionError: View function mapping is overwriting an existing endpoint function: view_chat_data